In [1]:
import os
import pandas as pd
from scipy import stats
from datetime import datetime
import numpy as np
import traceback

# pd.set_option('display.max_columns', None) 
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

Load LifeLines and REDCap variables

In [2]:
#Lifelines
path_high=os.getcwd()+"/high_BMI_lifelines.csv"
path_low=os.getcwd()+"/low_BMI_lifelines.csv"
path_noage=os.getcwd()+"/LifeLines_BMI_export_latest.csv"

#Redcap
path_redcap=os.getcwd()+"/BMI_6-6-2023.csv"
path_gender=os.getcwd()+"/BMI_all_participants_with_filled_scan_tab.csv"

In [3]:
high_BMI=pd.read_csv(path_high)
high_BMI

,patientID,weight_at_scan,length_at_scan,first_scan_date,age_at_scan,gender,bp_tlv,bmi,age_5yr,age_10yr,...,pack_year_categories,copd_diagnosis,asthma_diagnosis,resp_other,fev1,fvc,fev1_pp,fev1fvc_lln,fev1_fvc,GOLD_stage
0,101097,107.0,1.610,02/10/2017,58.0,Female,4.698,41.279,55-60,50-60,...,10-20,False,False,WHEEZE,2.35,3.23,91.17,0.68,0.728,0
1,101580,112.0,1.735,27/09/2017,45.0,Female,4.741,37.207,45-50,40-50,...,10-20,False,False,False,2.63,3.13,77.01,0.70,0.840,0
2,102007,105.0,1.640,23/09/2017,55.0,Female,5.004,39.039,55-60,50-60,...,1-10,False,False,False,2.62,3.60,94.80,0.69,0.728,0
3,102965,128.0,1.670,30/10/2017,59.0,Female,4.471,45.896,55-60,50-60,...,1-10,False,True,WHEEZE,2.38,3.01,85.51,0.68,0.791,0
4,103302,115.0,1.700,12/10/2017,50.0,Female,5.405,39.792,50-55,50-60,...,20+,True,False,WHEEZE,2.46,3.35,78.83,0.70,0.734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,970898,125.0,1.725,04/04/2019,53.0,Female,3.956,42.008,50-55,50-60,...,0,False,False,False,2.64,3.47,83.76,0.69,0.761,0
176,980529,112.0,1.610,28/11/2019,67.0,Female,4.414,43.208,65-70,60-70,...,1-10,False,False,False,2.00,2.72,83.96,0.67,0.735,0
177,986867,122.0,1.810,17/11/2020,64.0,Male,5.809,37.239,60-65,60-70,...,1-10,False,False,False,3.06,3.77,80.45,0.66,0.812,0
178,990458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
low_BMI['patientID']

0      100025
1      100094
2      101122
3      101479
4      101493
        ...  
175    983431
176    987624
177    995595
178    997268
179    997831
Name: patientID, Length: 180, dtype: int64

In [26]:
low_BMI=pd.read_csv(path_low)
low_BMI=low_BMI[~low_BMI['patientID'].isin([130781,334327])] #Exclude cases with 10 nods in first reading
low_BMI

,patientID,weight_at_scan,length_at_scan,first_scan_date,age_at_scan,gender,bp_tlv,bmi,age_5yr,age_10yr,...,pack_year_categories,copd_diagnosis,asthma_diagnosis,resp_other,fev1,fvc,fev1_pp,fev1fvc_lln,fev1_fvc,GOLD_stage
0,100025,58.0,1.765,29/08/2017,49.0,Female,4.655,18.618,45-50,40-50,...,0,False,False,False,2.90,3.34,85.51,0.69,0.868,0
1,100094,57.0,1.740,09/09/2017,53.0,Female,5.653,18.827,50-55,50-60,...,0,False,False,False,3.08,3.76,95.87,0.69,0.819,0
2,101122,61.0,1.770,04/10/2017,48.0,Female,5.977,19.471,45-50,40-50,...,0,False,True,WHEEZE,3.55,4.87,102.68,0.70,0.729,0
3,101479,55.0,1.680,11/11/2017,54.0,Female,5.098,19.487,50-55,50-60,...,1-10,False,False,False,2.73,3.57,NaN,NaN,0.765,0
4,101493,55.0,1.740,21/09/2017,55.0,Female,5.621,18.166,55-60,50-60,...,0,False,False,False,3.59,4.21,114.12,0.69,0.853,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,983431,65.0,1.820,13/12/2021,77.0,Male,5.857,19.623,75-80,70-80,...,10-20,False,False,False,2.98,4.33,83.98,0.63,0.688,0
176,987624,61.0,1.770,02/04/2022,84.0,Male,7.761,19.471,80-85,80-90,...,1-10,False,False,False,4.20,5.76,135.10,0.61,0.729,0
177,995595,46.0,1.530,09/04/2022,75.0,Female,3.743,19.651,75-80,70-80,...,0,False,False,False,1.97,2.73,94.87,0.66,0.722,0
178,997268,59.0,1.770,13/06/2019,56.0,Female,4.824,18.832,55-60,50-60,...,10-20,False,False,False,2.54,3.50,78.43,0.68,0.726,0


In [5]:
BMI_lifelines=pd.concat([high_BMI,low_BMI])

In [6]:
BMI_red=pd.read_csv(path_redcap)
BMI_red=BMI_red[BMI_red['participant_id'].isin(list(high_BMI['patientID'].values)+ list(low_BMI['patientID'].values))]

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3106: DtypeWarning: Columns (17,18,39,48,58,90,91,127,164,201,238,241,242,275,349) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
BMI_gender=pd.read_csv(path_gender)
BMI_gender=BMI_gender[BMI_gender['participant_id'].isin(list(high_BMI['patientID'].values)+ list(low_BMI['patientID'].values))]

assert np.sum(BMI_gender['participant_gender___1']==1)+np.sum(BMI_gender['participant_gender___2']==1)==len(BMI_gender) #1 male, 2 female

In [8]:
BMI_gender['participant_gender___1']=BMI_gender['participant_gender___1'].map({1:'Male', 0:'Female'})
BMI_gender.rename(columns={'participant_gender___1': 'gender'}, inplace=True)
BMI_gender

,participant_id,participant_last_name,date_of_birth,gender,participant_gender___2,participant_weight,participant_length
0,100025,ImaLife_100025,1967-10-29,Female,1,58.0,176.5
7,100094,ImaLife_100094,1964-08-10,Female,1,57.0,174.0
101,101097,ImaLife_101097,1958-11-05,Female,1,107.0,161.0
104,101122,ImaLife_101122,1969-04-23,Female,1,61.0,177.0
139,101479,ImaLife_101479,1962-12-28,Female,1,55.0,168.0
...,...,...,...,...,...,...,...
10865,990458,ImaLife_990458,1958-07-20,Female,1,112.0,166.5
10902,995595,ImaLife_995595,1946-08-31,Female,1,46.0,153.0
10903,995606,ImaLife_995606,1946-04-04,Male,0,111.0,173.0
10914,997268,ImaLife_997268,1962-09-06,Female,1,59.0,177.0


In [9]:
BMI_gender_high=BMI_gender[BMI_gender['participant_id'].isin(list(high_BMI['patientID'].values))]
BMI_gender_low=BMI_gender[BMI_gender['participant_id'].isin(list(low_BMI['patientID'].values))]

Calculate age for LifeLines and REDCap and compare them

In [10]:
# #Function to calculate age of participant from age in which he/she was born
def calculate_age(scan_date, birth_date):
    try:
        birth_date=datetime(int(birth_date[:4]),int(birth_date[5:7]),int(birth_date[8:10]),0,0)
        scan_date=datetime(int(scan_date[:4]),int(scan_date[5:7]),int(scan_date[8:10]),0,0)
        return int(scan_date.year - birth_date.year - ((scan_date.month, scan_date.day) < (birth_date.month, birth_date.day)))
    except:
        return np.nan

In [11]:
BMI_red['age_at_scan']=BMI_red.apply(lambda row: calculate_age(row['scan_date'],row['date_of_birth']), axis=1)

In [12]:
BMI_red_high=BMI_red[BMI_red['participant_id'].isin(high_BMI['patientID'])]
BMI_red_low=BMI_red[BMI_red['participant_id'].isin(low_BMI['patientID'])]

In [13]:
#High BMI
total_null=0
for index,age in enumerate(high_BMI['age_at_scan'].values):
    if age==BMI_red_high['age_at_scan'].iloc[index]:
        pass
    else:
        if age<200: #Ensure no nan values are printed
            print('From RedCap scan date is',BMI_red_high['scan_date'].iloc[index], 'and date of birth is',BMI_red_high['date_of_birth'].iloc[index])
            print("RedCap age is:",calculate_age(BMI_red_high['scan_date'].iloc[index],BMI_red_high['date_of_birth'].iloc[index]))
            print('From LifeLines age at scan is',age)
            print('Participant is:',high_BMI['patientID'].iloc[index])
            print('\n')
        else:
            total_null+=1
        #     # print('From RedCap scan date is',BMI_red_high['scan_date'].iloc[index], 'and date of birth is',BMI_red_high['date_of_birth'].iloc[index])
        #     # print('From LifeLines age at scan is',age)
        #     print('Nan values in LifeLines data for participant',high_BMI['patientID'].iloc[index])
        #     print('\n')

print('Total null values in LifeLines data:',total_null)
assert high_BMI.isnull()['age_at_scan'].sum()==total_null

From RedCap scan date is 2020-04-30 and date of birth is 1967-06-22
RedCap age is: 52
From LifeLines age at scan is 50.0
Participant is: 103302


From RedCap scan date is 2020-06-07 and date of birth is 1967-06-23
RedCap age is: 52
From LifeLines age at scan is 50.0
Participant is: 108772


From RedCap scan date is 2020-06-07 and date of birth is 1969-12-19
RedCap age is: 50
From LifeLines age at scan is 47.0
Participant is: 112498


Total null values in LifeLines data: 22


In [14]:
#Low BMI
total_null=0
for index,age in enumerate(low_BMI['age_at_scan'].values):
    
    if age==BMI_red_low['age_at_scan'].iloc[index]:
        pass
    else:
        if age<200: #Ensure no nan values are printed
            print('From RedCap scan date is',BMI_red_low['scan_date'].iloc[index], 'and date of birth is',BMI_red_low['date_of_birth'].iloc[index])
            print("RedCap age is:",calculate_age(BMI_red_low['scan_date'].iloc[index],BMI_red_low['date_of_birth'].iloc[index]))
            print('From LifeLines age at scan is',age)
            print('Participant is:',low_BMI['patientID'].iloc[index])
            print('\n')
        else:
            total_null+=1
        #     # print('From RedCap scan date is',BMI_red_high['scan_date'].iloc[index], 'and date of birth is',BMI_red_high['date_of_birth'].iloc[index])
        #     # print('From LifeLines age at scan is',age)
            # print('Nan values in LifeLines data for participant',low_BMI['patientID'].iloc[index])
        #     print('\n')

print('Total null values in LifeLines data:',total_null)
assert low_BMI.isnull()['age_at_scan'].sum()==total_null

From RedCap scan date is 2020-04-23 and date of birth is 1967-03-23
RedCap age is: 53
From LifeLines age at scan is 50.0
Participant is: 103906


From RedCap scan date is 2020-05-07 and date of birth is 1962-07-21
RedCap age is: 57
From LifeLines age at scan is 55.0
Participant is: 105600


Total null values in LifeLines data: 9


Get rest of variables from REDCap - Copied from 'demographics_emphysema_statistics.ipynb'

In [15]:
BMI_red

,participant_id,participant_last_name,date_of_birth,participant_gender___1,participant_gender___2,participant_weight,participant_length,perm_date,share_siemens,share_studies,...,ai_detect_nod_n10,ai_true_nod_n10,ai_id_nod_n10,ai_vol_nod_n10,ai_seg_cor_n10,ai_dia2d_nod_n10,ai_dia3d_nod_n10,gen_rem_nod,nodule_quantification_first_scan_complete,age_at_scan
2,100025,ImaLife_100025,1967-10-29,0,1,58.0,176.5,2017-07-28,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,49
9,100094,ImaLife_100094,1964-08-10,0,1,57.0,174.0,2017-07-31,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,53
109,101097,ImaLife_101097,1958-11-05,0,1,107.0,161.0,2017-07-29,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,58
112,101122,ImaLife_101122,1969-04-23,0,1,61.0,177.0,2017-07-28,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,48
147,101479,ImaLife_101479,1962-12-28,0,1,55.0,168.0,2017-08-05,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13012,990458,ImaLife_990458,1958-07-20,0,1,112.0,166.5,2019-04-16,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,60
13065,995595,ImaLife_995595,1946-08-31,0,1,46.0,153.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,75
13066,995606,ImaLife_995606,1946-04-04,1,0,111.0,173.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,75
13081,997268,ImaLife_997268,1962-09-06,0,1,59.0,177.0,2019-04-22,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,56


In [16]:
#Check if there are any missing values
age=BMI_red['age_at_scan']
weight=BMI_red['participant_weight']
height=BMI_red['participant_length']

gender=BMI_gender['gender']
never_smoker=BMI_lifelines['never_smoker']
ever_smoker=BMI_lifelines['ever_smoker']
current_smoker=BMI_lifelines['current_smoker']
ex_smoker=BMI_lifelines['ex_smoker']
pack_years=BMI_lifelines['pack_years']

try: #Confirm that there are no missing values for age, weight, height
    assert np.sum(age.value_counts().values)==len(BMI_red)
    assert np.sum(weight.value_counts().values)==len(BMI_red)
    assert np.sum(height.value_counts().values)==len(BMI_red)
except:
    print(traceback.format_exc())
    print('\n')

#Missing values for gender
try:
    assert np.sum(gender.value_counts().values)==len(BMI_lifelines)
except:
    print("For gender we have {} values out of {}".format(np.sum(gender.value_counts().values),len(BMI_lifelines)))

#Report missing values for smoking attributes
try:
    assert np.sum(never_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For never_smoker we have {} values out of {}".format(np.sum(never_smoker.value_counts().values),len(BMI_lifelines)))

try:
    assert np.sum(ever_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For ever_smoker we have {} values out of {}".format(np.sum(ever_smoker.value_counts().values),len(BMI_lifelines)))
    
try:
    assert np.sum(current_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For current_smoker we have {} values out of {}".format(np.sum(current_smoker.value_counts().values),len(BMI_lifelines)))
    
try:
    assert np.sum(ex_smoker.value_counts().values)==len(BMI_lifelines)
except:
    print("For ex_smoker we have {} values out of {}".format(np.sum(ex_smoker.value_counts().values),len(BMI_lifelines)))
    
try:
    assert np.sum(pack_years.value_counts().values)==len(BMI_lifelines)
except:
    print("For pack_years we have {} values out of {}".format(np.sum(pack_years.value_counts().values),len(BMI_lifelines)))

For never_smoker we have 329 values out of 360
For ever_smoker we have 329 values out of 360
For current_smoker we have 329 values out of 360
For ex_smoker we have 329 values out of 360
For pack_years we have 179 values out of 360


In [17]:
#Create df with demographics to be filled below for each of the emphysema/non-emphysema categories
df_statistics=pd.DataFrame(columns=['All participants','Participants with low BMI','Participants with high BMI','P value'],
                            index=['Number of participants','Gender Male', 'Gender Female',
                                   'Age', 'Weight', 'Height',
                                   'Never smoker','Ever smoker','Pack years']) #'Current smoker', 'Ex smoker',

df_statistics.index.name = 'Characteristics' 

#All participants statistics
num_participants=len(BMI_red) #Same as BMI_lifelines
males=np.sum(BMI_gender['gender']=='Male')
females=np.sum(BMI_gender['gender']=='Female')
age_mean=np.mean(BMI_red['age_at_scan'])
age_std=np.std(BMI_red['age_at_scan'])
weight_mean=np.mean(BMI_red['participant_weight'])
weight_std=np.std(BMI_red['participant_weight'])
length_mean=np.mean(BMI_red['participant_length'])
length_std=np.std(BMI_red['participant_length'])

never_smoker=np.sum(BMI_lifelines['never_smoker']==True)
ever_smoker=np.sum(BMI_lifelines['ever_smoker']==True)
current_smoker=np.sum(BMI_lifelines['current_smoker']==True)
ex_smoker=np.sum(BMI_lifelines['ex_smoker']==True)
pack_years_mean=np.mean(BMI_lifelines['pack_years'])
pack_years_std=np.std(BMI_lifelines['pack_years'])

#High BMI participants statistics
num_participants_high=len(BMI_red_high)
males_high=np.sum(BMI_gender_high['gender']=='Male')
females_high=np.sum(BMI_gender_high['gender']=='Female')
age_mean_high=np.mean(BMI_red_high['age_at_scan'])
age_std_high=np.std(BMI_red_high['age_at_scan'])
weight_mean_high=np.mean(BMI_red_high['participant_weight'])
weight_std_high=np.std(BMI_red_high['participant_weight'])
length_mean_high=np.mean(BMI_red_high['participant_length'])
length_std_high=np.std(BMI_red_high['participant_length'])

never_smoker_high=np.sum(high_BMI['never_smoker']==True)
ever_smoker_high=np.sum(high_BMI['ever_smoker']==True)
current_smoker_high=np.sum(high_BMI['current_smoker']==True)
ex_smoker_high=np.sum(high_BMI['ex_smoker']==True)
pack_years_mean_high=np.mean(high_BMI['pack_years'])
pack_years_std_high=np.std(high_BMI['pack_years'])

#Low BMI participants statistics
num_participants_low=len(BMI_red_low)
males_low=np.sum(BMI_gender_low['gender']=='Male')
females_low=np.sum(BMI_gender_low['gender']=='Female')
age_mean_low=np.mean(BMI_red_low['age_at_scan'])
age_std_low=np.std(BMI_red_low['age_at_scan'])
weight_mean_low=np.mean(BMI_red_low['participant_weight'])
weight_std_low=np.std(BMI_red_low['participant_weight'])
length_mean_low=np.mean(BMI_red_low['participant_length'])
length_std_low=np.std(BMI_red_low['participant_length'])

never_smoker_low=np.sum(low_BMI['never_smoker']==True)
ever_smoker_low=np.sum(low_BMI['ever_smoker']==True)
current_smoker_low=np.sum(low_BMI['current_smoker']==True)
ex_smoker_low=np.sum(low_BMI['ex_smoker']==True)
pack_years_mean_low=np.mean(low_BMI['pack_years'])
pack_years_std_low=np.std(low_BMI['pack_years'])


#Add them to df
df_statistics['All participants']=[
                        num_participants,
                        str(males)+' ('+str(100*np.round(males/num_participants,2))+'%)',
                        str(females)+' ('+str(100*np.round(females/num_participants,2))+'%)',
                        str(np.round(age_mean,1))+str('±')+str(np.round(age_std,1)),
                        str(np.round(weight_mean,1))+str('±')+str(np.round(weight_std,1)),
                        str(np.round(length_mean,1))+str('±')+str(np.round(length_std,1)),
                        str(never_smoker)+' ('+str(100*np.round(never_smoker/num_participants,2))+'%)', 
                        str(ever_smoker)+' ('+str(100*np.round(ever_smoker/num_participants,2))+'%)',
                        str(np.round(pack_years_mean,1))+str('±')+str(np.round(pack_years_std,1))]

df_statistics['Participants with high BMI']=[
                        str(num_participants_high)+' ('+str(100*np.round(num_participants_high/num_participants,2))+'%)',
                        str(males_high)+' ('+str(100*np.round(males_high/num_participants_high,2))+'%)',
                        str(females_high)+' ('+str(100*np.round(females_high/num_participants_high,2))+'%)',
                        str(np.round(age_mean_high,1))+str('±')+str(np.round(age_std_high,1)),
                        str(np.round(weight_mean_high,1))+str('±')+str(np.round(weight_std_high,1)),
                        str(np.round(length_mean_high,1))+str('±')+str(np.round(length_std_high,1)),
                        str(never_smoker_high)+' ('+str(100*np.round(never_smoker_high/num_participants_high,1))+'%)', 
                        str(ever_smoker_high)+' ('+str(100*np.round(ever_smoker_high/num_participants_high,2))+'%)', 
                        str(np.round(pack_years_mean_high,1))+str('±')+str(np.round(pack_years_std_high,1))]

df_statistics['Participants with low BMI']=[
                        str(num_participants_low)+' ('+str(100*np.round(num_participants_low/num_participants,1))+'%)',
                        str(males_low)+' ('+str(np.round(100*males_low/num_participants_low,1))+'%)',
                        str(females_low)+' ('+str(np.round(100*females_low/num_participants_low,1))+'%)',       
                        str(np.round(age_mean_low,1))+str('±')+str(np.round(age_std_low,1)),
                        str(np.round(weight_mean_low,1))+str('±')+str(np.round(weight_std_low,1)),
                        str(np.round(length_mean_low,1))+str('±')+str(np.round(length_std_low,1)),
                        str(never_smoker_low)+' ('+str(np.round(100*never_smoker_low/num_participants_low,1))+'%)', 
                        str(ever_smoker_low)+' ('+str(np.round(100*ever_smoker_low/num_participants_low,1))+'%)',
                        str(np.round(pack_years_mean_low,1))+str('±')+str(np.round(pack_years_std_low,1))]

df_statistics
#Symbol '±' copied from online
#'ever smoker' is 'current smoker'+'ex smoker'. 

,All participants,Participants with low BMI,Participants with high BMI,P value
Characteristics,,,,
Number of participants,360,180 (50.0%),180 (50.0%),NaN
Gender Male,68 (19.0%),25 (13.9%),43 (24.0%),NaN
Gender Female,292 (81.0%),155 (86.1%),137 (76.0%),NaN
Age,59.8±10.3,61.3±10.8,58.2±9.5,NaN
Weight,85.7±32.9,55.0±6.3,116.5±15.2,NaN
Height,170.9±8.5,171.2±8.3,170.6±8.8,NaN
Never smoker,138 (38.0%),82 (45.6%),56 (30.0%),NaN
Ever smoker,191 (53.0%),89 (49.4%),102 (56.99999999999999%),NaN
Pack years,15.1±14.1,15.1±13.1,15.0±14.9,NaN


In [18]:
#Manual correction of rounding errors
df_statistics['Participants with high BMI'][-2]=df_statistics['Participants with high BMI'][-2][:4]+'('+ str(
    np.round(float(df_statistics['Participants with high BMI'][-2][5:10])))+'%)'


### P-value calculations

In [19]:
P_vals=['','gender=','','age=','weight=','height=','smoking status=','','pack years='] #For those column we calculate p values

In [20]:
#Create two groups of categorical data
group1 = [males_high, males_low]  # frequencies of each category in group 1 - males
group2 = [females_high, females_low]  # frequencies of each category in group 2 - females

# calculate the chi-square statistic and p-value
chi2_statistic, p_value, dof, expected = stats.chi2_contingency([group1, group2])

# print the p-value
print("P-value for gender:", p_value)
P_vals[1]=str(p_value)


# create two groups of numeric data
group1 = BMI_red_low['age_at_scan']
group2 = BMI_red_high['age_at_scan']

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for age:", p_value)
P_vals[3]=str(p_value)


#Repeat above for other categorical and numeric data

# create two groups of numeric data
group1 = BMI_red_low['participant_weight']
group2 = BMI_red_high['participant_weight']

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for weight:", p_value)
P_vals[4]=str(p_value)


# create two groups of numeric data
group1 = BMI_red_low['participant_length']
group2 = BMI_red_high['participant_length']

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for length:", p_value)
P_vals[5]=str(p_value)

# create two groups of numeric data
group1 = low_BMI['pack_years'].dropna()
group2 = high_BMI['pack_years'].dropna()

# calculate the t-statistic and p-value
t_statistic, p_value = stats.ttest_ind(group1, group2)

# print the p-value
print("P-value for pack_years:", p_value)
P_vals[8]=str(p_value) #was 10


# create two groups of categorical data
group1 = [never_smoker_high, never_smoker_low]  # frequencies of each category in group 1 - never smokers
group2 = [ever_smoker_high, ever_smoker_low]  # frequencies of each category in group 2 - even smokers

# calculate the chi-square statistic and p-value
chi2_statistic, p_value, dof, expected = stats.chi2_contingency([group1, group2])

# print the p-value
print("P-value for smoking status (never, ever):", p_value)
P_vals[6]=str(p_value)

P-value for gender: 0.022076750416850692
P-value for age: 0.0036037533060907802
P-value for weight: 5.7811653185586496e-164
P-value for length: 0.5309493873438867
P-value for pack_years: 0.9598155061679055
P-value for smoking status (never, ever): 0.02884747870070215


In [21]:
#P values smaller than 0.001 set to '<0.001' and others are rounded to 2 digits
for ind,val in enumerate(P_vals):
    if val!='':
        if float(val)<0.001:
            P_vals[ind]='<0.001'
        else:
            P_vals[ind]=np.round(float(val),3)
    else:
        pass
    
df_statistics['P value']=P_vals
df_statistics

,All participants,Participants with low BMI,Participants with high BMI,P value
Characteristics,,,,
Number of participants,360,180 (50.0%),180 (50.0%),
Gender Male,68 (19.0%),25 (13.9%),43 (24.0%),0.022
Gender Female,292 (81.0%),155 (86.1%),137 (76.0%),
Age,59.8±10.3,61.3±10.8,58.2±9.5,0.004
Weight,85.7±32.9,55.0±6.3,116.5±15.2,<0.001
Height,170.9±8.5,171.2±8.3,170.6±8.8,0.531
Never smoker,138 (38.0%),82 (45.6%),56 (30.0%),0.029
Ever smoker,191 (53.0%),89 (49.4%),102 (57.0%),
Pack years,15.1±14.1,15.1±13.1,15.0±14.9,0.96


In [23]:
df_statistics.to_excel('demographics_BMI_statistics.xlsx')